# Udacity Machine Learning Nanodegree Capstone Project

## Recurrent Neural Network based language model.

### Naren Doraiswamy

#### August 2017


The traditional neural networks have given some great results after the evolution of the kind of computational efficiency that is available today and the huge amounts of data that can be processed with this computational power. These traditional networks just take in the inputs without assuming any dependencies between them and this might be a problem when we are working on problems like natural language processing , speech/audio processing and also in vision problems where adjacent pixels are almost the same.


In this particular capstone project ,i will be working on the language processing problem where i will train a network on a particular kind of text and then try to generate similar kind of text from the model.


In this particular capstone project ,i will be working on the language processing problem where i will train a network on a particular kind of text and then try to generate similar kind of text from the model.


The papers that i have referred are given below:

[Language Model based on Recurrent Neural Network](http://www.fit.vutbr.cz/research/groups/speech/publi/2010/mikolov_interspeech2010_IS100722.pdf)

[Extensions of recurrent neural network language model](http://www.fit.vutbr.cz/research/groups/speech/publi/2011/mikolov_icassp2011_5528.pdf)

[Generating Text with Recurrent Neural Networks](http://machinelearning.wustl.edu/mlpapers/paper_files/ICML2011Sutskever_524.pdf)


Language Modeling using LSTM.
Our goal is to build a Language Model using a Recurrent Neural Network. Let's say we have sentence of n words. Language Model allows us to predict the probability of observing the sentence (in a given dataset) as: P(w1,...,wn)=∏i=1nP(wi∣w1,...,wi−1)P(w1,...,wn)=∏i=1nP(wi∣w1,...,wi−1) In words, the probability of a sentence is the product of probabilities of each word given the words that came before it. So, the probability of the sentence "He went to buy some chocolate" would be the probability of "chocolate" given "He went to buy some", multiplied by the probability of "some" given "He went to buy", and so on. Why is that useful? Why would we want to assign a probability to observing a sentence? First, such a model can be used as a scoring mechanism. For example, a Machine Translation system typically generates multiple candidates for an input sentence. You could use a language model to pick the most probable sentence. Intuitively, the most probable sentence is likely to be grammatically correct. Similar scoring happens in speech recognition systems. But solving the Language Modeling problem also has a cool side effect. Because we can predict the probability of a word given the preceding words, we are able to generate new text. It's a generative model. Given an existing sequence of words we sample a next word from the predicted probabilities, and repeat the process until we have a full sentence. And this is exactly what we are going to do i.e: Generate new text


We will be using a special case of RNN's called LSTM since vanilla RNN's cannot hold on to the memory for a long time and suffers from Vanishing and Exploding gradient problem. Its just a simple manipulation of vanilla RNN and there are many different variants in Rnn's , the other famous one being gated recurrent units GRU
So lets get started with the implementation

In [1]:
import numpy as np
import tensorflow as tf
import time
import sys

In [2]:
# Open data file and convert to lowercase
filename = 'GrimmsFairyTales.txt'
text =  open(filename).read().lower()

print('text length:', len(text))

# create and sort a list of unique characters
chars = sorted(list(set(text)))
nb_chars = len(chars)
print('number of unique chars:', len(chars))

text length: 517631
number of unique chars: 49


In [3]:
print (chars)

['\n', ' ', '!', '(', ')', '*', ',', '-', '.', '1', '2', '3', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '“', '”', '\ufeff']


### Data Exploration

In [4]:
view_sentence_range = (0, 10)
print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
paras = text.split('\n\n')
print('Number of paragraphs: {}'.format(len(paras)))


sentences = [sentence for para in paras for sentence in para.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 9699
Number of paragraphs: 1040
Number of lines: 8038
Average number of words in each line: 12.522144812142324

The sentences 0 to 10:
﻿


the golden bird

a certain king had a beautiful garden, and in the garden stood a tree
which bore golden apples. these apples were always counted, and about
the time when they began to grow ripe it was found that every night one
of them was gone. the king became very angry at this, and ordered the
gardener to keep watch all night under the tree. the gardener set his


### Pre-processing

Since our input is text and we cannot just input them to our model as it understands only numbers and not text, we'll load the text file and convert it into integers for our network to use. We will create a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [5]:
# convert each unique character to an integer and vice versa
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Okie, we already had our text and a "dictionary" of unique characters that we will use to generate text from. Next, we need to standardize the input sequences that we will feed into our model. We will need our sequences to has same lengths. This can be done by predefining a fixed length window that we will use to slide through our text by one step at a time. Each slide will create one sequence in our input. In fact, we can even adjust our slideing step.

In [7]:
fixlen = 40
step = 1
X_train = []
y_train = []
for i in range(0, len(text) - fixlen, step):
    X_train.append(text[i: i + fixlen])
    y_train.append(text[i + fixlen])
    
print ('number of input sequences:', len(X_train))
print ('number of output characters:', len(y_train))

number of input sequences: 517591
number of output characters: 517591


We see that we have almost the same number of imput sequence (due to our step =1). Correspondingly, we have the same number of output characters. But, again, we need to index characters into integers in order to work with Keras. This is called one hot encoding.
It is very simple. Remember we create a indexed "dictionary" before (named chars)? Now, each output character in y, we just convert it into 47-long vector with all of "0" and only one "1" for corresponding to the index of that character in the "dictionary". Let's look at an example. Let's see what character it is for y_train[88]:


In [8]:
y_train[88]

's'

So, it is "s". And what is indexed of s in our dictionary?

In [9]:
char_indices['s']

36

It is 36. Thus our one hot vector has all entries of "0" except for entry 36th, its value is "1". Now, we need to do this for all other values in y. First, we convert chars to integers, then we explode all integers to categorical (one hot) vectors. There are sevaral ways to explode categorical data, for simplicity, I will use a utility from Keras.

In [10]:
y = []
# Convert chars in output into integers corresponding with their indices in the dictionary
for i in range(0, len(y_train)):
    y.append(char_indices[y_train[i]])

# Expode all integer values in to one hot vectors    
from keras.utils import np_utils
y = np_utils.to_categorical(y)

Using TensorFlow backend.


Similarly, we should somehow convert our sequences in X_train data into integers. We will do it slightly different. For each (row) vector in X_train, we convert characters into its indices in our vocabulary. For example, let look at row 88.

In [11]:
X_train[88]

'ee\nwhich bore golden apples. these apple'

We wil convert it into a vector x.

In [12]:
x=[]
x.append([char_indices[char] for char in X_train[88]])
print (x)

[[22, 22, 0, 40, 25, 26, 20, 25, 1, 19, 32, 35, 22, 1, 24, 32, 29, 21, 22, 31, 1, 18, 33, 33, 29, 22, 36, 8, 1, 37, 25, 22, 36, 22, 1, 18, 33, 33, 29, 22]]


Now, let's do this for all the vectors in X_train and store it in X

In [13]:
X_=[]
for i in range(0, len(text) - fixlen, step):
    X_.append([char_indices[char] for char in X_train[i]])

Okie, we just had our text data converted into integers so that we can work with. But we still need to do a bit more to
(1) convert it into a shape that LSTM accept, that is [data sample, sequence length, sequence depth]
(2) normalize data to values between [0,1], remember that there is a sigmoid activation layer in LSTM? we do not want our number is too big when it is fed to sigmoid layer.

In [14]:
# We need numpy library to do the shaping
import numpy as np

# Reshape and normalize input data
X = np.reshape(X_,(len(X_),fixlen, 1))
X = X/float(nb_chars)

 Next, we will build up an LSTM network to play with our data.

### Building a LSTM Recurrent Neural Network

#### RNN basics 

Before we dive in to the code, let's understand RNN and LSTM in general first. RNN is a type of neural network that allow to memorize information of the input for long time. To do this, it needs to have loops inside so that information can be stored. In picture, it looks [like this](http://www.deeplearningbook.org/contents/rnn.html).

In the left is unfolded RNN, in the right is RRN which is unrolled to each step of the input sequence. Thus, by its nature, RNN is suitable to model sequential data such as time series, speech, text... which input from time t may depend on previous time steps (t-1,t-2,etc).
Well, for someone with control background, this reasoning of RNN momorization is no surprise. Indeed, RNN can be thought as a discrete-time dynamical systems which, in control, is modeled by difference equation:
x(k+1) = f(x(k),u(k))
h(k) = g(x(k),u(k))
where x(k) is state, u(k) is input, and h(k) is output at time k. Function f is called dynamics, g is output map. We know that state variables are enough to summarize all information in the past of a systems and that explain why RNN can memmorize information for long time.

### Problems with RNN

If we think of RNN as a discrete-time dynamical system, from stability theory point of view, we can expect that the state of RNN could 1. settling down to an equilibrium or 2. explode to infinity and 3. very rare, state can oscillate.
Another problem is that we will use backprop to train our RNN. Backprop is in fact a method to calculate gradient by chain rule. This lead to problem with multiple multiplications of gradient matrices during backprop. The consequence is that the gradient will become smaller and smaller or worse become biger and exploded. Thus, RNN has two intrisic problems: vanishing gradient and exploding gradient. In either case, all learned information will be lost.
One way to overcome it is long short term memory (LSTM) network.

### LSTM

There is an excellent explaination about the struture of LSTM in this [link](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). I will not going to the details but in general, LSTM use "gates" to direct information flow within the network cell that will decide which information to discard or allow to flow through.
Well, now let's play with LSTM networks.

### Building the network

In [17]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint
import random
import sys


# Building LSTM model

model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.5))
model.add(Dense((nb_chars-1), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')


# Set a checkpoint to save the best model
model_saving = "weight_saving_at_{epoch:01d}_and_{loss: 0.4f}.hdf5"
checkpoint =  ModelCheckpoint(model_saving, monitor = 'loss', verbose =1, save_best_only = False, mode = 'min')
callbacks = [checkpoint]

### Metrics:
Since the output layer is a fully connected layer which implies the probabilities of the next character to be outputted, we will use the softmax classifier along with the categorical cross entropy for loss generation during the training. So we will one hot encode our targets, and compare them with the generated probabilities of the output characters and use the formulae y*ln(y_). Since we are using the LSTM’s instead of the vanilla RNN’s , the problem of vanishing gradients is taken care of, but the exploding gradients problem still persists. Hence we will use the gradient clipping method which will have a threshold where anything beyond this value will clip it to the threshold. The LSTM is trained with mini-batch Stochastic Gradient Descent and I have used Adam optimizer(per-parameter adaptive learning rate methods) to stablilize the updates.Also RMSProp can also be used though adam is more adaptive in nature.

### Hyperparameters

batch_size - Number of sequences running through the network in one pass.

num_steps - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.

lstm_size - The number of units in the hidden layers.

num_layers - Number of hidden LSTM layers to use

learning_rate - Learning rate for training

keep_prob - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

### Parameters discussion:

The model is pretty robust enough with the above used parameter, though I would like to use the batch size of 32/64/128/256 and train the network on more epochs. This would get better results but since the trial pack on Floyd-hub doesn’t allow networks to run above an hour and hence I have chosen a longer batch_size of 500(not really advisable, see the best strategy below) and the sequence step of 50. So the model can be trusted upon if these changes are adapted in the training and can be deployed on a large scale, though these parameters are reasonable enough and aligns with our solution expectations.

### Benchmark

The [gihub repo](https://github.com/udacity/deep-learning/tree/master/intro-to-rnns) is taken as reference/benchmark while i started. The implementation set a particular benchmark loss after being trained for a long time,but the trial account on the floydhub limits the GPU usage upto an hour. The loss can be reduced further , if the network is trained for a longer time, and produce a more robust and generative model. The result obtained here are pretty satisfying though and produces similar results like the one considered as the benchmark. 

### Training the network

In [19]:
nb_epoch = 5
batch_size = 1000
model.fit(X,y, nb_epoch = nb_epoch, batch_size = batch_size, callbacks = callbacks)

/usr/local/lib/python3.5/site-packages/keras/models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/5
517591/517591 [==============================] - 428s - loss: 2.5847   
Epoch 2/5
517591/517591 [==============================] - 429s - loss: 2.3372   
Epoch 3/5
517591/517591 [==============================] - 429s - loss: 2.1403   
Epoch 4/5
517591/517591 [==============================] - 429s - loss: 1.9951   
Epoch 5/5
517591/517591 [==============================] - 428s - loss: 1.8872   


### Text Generation

Text Generation Now, after running several epoches, we can load a trained model to see how the network generates texts. I had here a saved mode weight_saving_at_003and 1.8215.hdf5 . This model is saved after 4 epoches and the loss was 1.8215. Let see how it generates texts.

In [20]:
'''load the trained weights'''
weights = "weight_saving_at_4_and_ 1.8872.hdf5"
#weights = "weight_saving_at_003_and_ 1.8215.hdf5"
model.load_weights(weights)
model.compile(loss='categorical_crossentropy', optimizer='adam')
start = np.random.randint(0, len(X)-1)
pattern = X_[start]
print (''.join([indices_char[value] for value in pattern]))
# generate characters
for i in range(150):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(nb_chars)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = indices_char[index]
    seq_in = [indices_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nEnd.")

re thou hangest!’

and the head answered
 to the woung was to the wood and said: ‘i will soon toon the wound be a gord to the wood and said, ‘i will soon to the wound be a gord to the wood an
End.


### Conclusion:

Now that we have seen the results can be really impressive with the RNN’s, especially with the LSTM's tweak. So after every epoch is completed, the checkpoint file which holds the weights at that particular instant will be saved in to the repository. Using every file , we see an improvement in the prediction of the words. initially the word spaces In the first checkpoint iteration we see that the network just learns the word spaces and then later on it tries to generate sentences with vague virtuoso and then in the next iteration it learns some most used words liker the he , she, its etc.. and now more english like text appears… I have used the checkpoint file from the 5 epoch and these weights are used. and if it is trained for a longer duration with lesser batch size it will be able to generate The punctuations, quotations, names etc in a very seemeingly natural way... So basically the model first discovers the general word space and then starts to rapidly learn the actual words. And if trained longer , the longer words can be generated too along with the theme of the training set. But we need enormous amount of data and very good computational efficiency to achieve that.

Also i used another small dataset as reddit dataset was too huge for the model and an epoch took more than an hour. So used a smaller dataset which convincingly gave out really good results.


__Also since this is a textual context that we are dealing with, there couldn't be any possible visualizations that could be shown though i found Andrew karpathy's blog really helpful in effective understanding of RNN's__